# Import development libraries

In [16]:
import bw2data as bd
import bw2calc as bc
import bw_processing as bwp
import numpy as np
import matrix_utils as mu

# Create new project

In [2]:
bd.projects.set_current("Multifunctionality")

Our existing implementation allows us to distinguish activities and prodducts, though not everyone does this.

In [3]:
db = bd.Database("background")
db.write({
    ("background", "1"): {
        "type": "process",
        "name": "1",
        "exchanges": [{
            "input": ("background", "bio"),
            "amount": 1,
            "type": "biosphere",
        }]
    }, 
    ("background", "2"): {
        "type": "process",
        "name": "2",
        "exchanges": [{
            "input": ("background", "bio"),
            "amount": 10,
            "type": "biosphere",
        }]
    },
    ("background", "bio"): {
        "type": "biosphere",
        "name": "bio",
        "exchanges": [],
    },
    ("background", "3"): {
        "type": "process",
        "name": "2",
        "exchanges": [
            {
                "input": ("background", "1"),
                "amount": 2,
                "type": "technosphere",
            }, {
                "input": ("background", "2"),
                "amount": 4,
                "type": "technosphere",
            }, {
                "input": ("background", "4"),
                "amount": 1,
                "type": "production",
                
            }
        ]
    },
    ("background", "4"): {
        "type": "product",
    }
})

Writing activities to SQLite3 database:
0% [#####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 02/18/2021 10:54:06
  Finished: 02/18/2021 10:54:06
  Total time elapsed: 00:00:00
  CPU %: 48.00
  Memory %: 0.69


In [39]:
method = bd.Method(("something",))
method.write([(("background", "bio"), 1)])

# LCA of background system

This database is fine and normal. It work the way we expect.

Here we use the preferred calling convention for Brightway 2.5, with the convenience function `prepare_lca_inputs`.

In [5]:
fu, data_objs, _ = bd.prepare_lca_inputs(demand={("background", "4"): 1}, method=("something",))

In [6]:
lca = bc.LCA(fu, data_objs=data_objs)
lca.lci()
lca.lcia()
lca.score

42.0

# Multifunctional activities

What happens when we have an activity that produces multiple products?

In [7]:
db = bd.Database("example mf")
db.write({
    # Activity
    ("example mf", "1"): {
        "type": "process",
        "name": "mf 1",
        "exchanges": [
            {
                "input": ("example mf", "2"),
                "amount": 2,
                "type": "production",
            }, {
                "input": ("example mf", "3"),
                "amount": 4,
                "type": "production",
            },
            {
                "input": ("background", "1"),
                "amount": 2,
                "type": "technosphere",
            }, {
                "input": ("background", "2"),
                "amount": 4,
                "type": "technosphere",
            }
        ]
    },
    # Product
    ("example mf", "2"): {
        "type": "good",
        "price": 4
    },
    # Product
    ("example mf", "3"): {
        "type": "good",
        "price": 6
    }
})

Writing activities to SQLite3 database:
0% [###] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 02/18/2021 10:54:07
  Finished: 02/18/2021 10:54:07
  Total time elapsed: 00:00:00
  CPU %: 18.50
  Memory %: 0.75


We can do an LCA of one of the products, but we will get a warning about a non-square matrix:

In [40]:
fu, data_objs, _ = bd.prepare_lca_inputs(demand={("example mf", "1"): 1}, method=("something",))

In [41]:
lca = bc.LCA(fu, data_objs=data_objs)
lca.lci()

NonsquareTechnosphere: Technosphere matrix is not square: 4 activities (columns) and 5 products (rows). Use LeastSquaresLCA to solve this system, or fix the input data

If we look at the technosphere matrix, we can see our background database (upper left quadrant), and the two production exchanges in the lower right:

In [42]:
lca.technosphere_matrix.toarray()

array([[ 1.,  0., -2., -2.],
       [ 0.,  1., -4., -4.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  2.],
       [ 0.,  0.,  0.,  4.]])

# Handling multifunctionality

There are many ways to do this. This notebook is an illustration of how such approaches can be madde easier using the helper libraries [bw_processing](https://github.com/brightway-lca/bw_processing) and [matrix_utils](https://github.com/brightway-lca/matrix_utils), not a statement that one approach is better (or even correct).

We create a new, in-memory "delta" `bw_processing` data package that gives new values for some additional columns in the matrix (the virtual activities generated by allocating each product), as well as updating values in the existing matrix.

In [11]:
def economic_allocation(dataset):
    assert isinstance(dataset, bd.backends.Activity)
    
    # Split exchanges into functional and non-functional
    functions = [exc for exc in dataset.exchanges() if exc.input.get('type') in {'good', 'waste'}]
    others = [exc for exc in dataset.exchanges() if exc.input.get('type') not in {'good', 'waste'}]
    
    for exc in functions:
        assert exc.input.get("price") is not None

    total_value = sum([exc.input['price'] * exc['amount'] for exc in functions])
        
    # Plus one because need to add (missing) production exchanges
    n = len(functions) * (len(others) + 1) + 1
    data = np.zeros(n)
    indices = np.zeros(n, dtype=bwp.INDICES_DTYPE)
    flip = np.zeros(n, dtype=bool)
    
    for i, f in enumerate(functions):
        allocation_factor = f['amount'] * f.input['price'] / total_value
        col = bd.get_id(f.input)
        
        # Add explicit production
        data[i * (len(others) + 1)] = f['amount']
        indices[i * (len(others) + 1)] = (col, col)

        for j, o in enumerate(others):
            index = i * (len(others) + 1) + j + 1
            data[index] = o['amount'] * allocation_factor
            flip[index] = o['type'] in {'technosphere', 'generic consumption'}
            indices[index] = (bd.get_id(o.input), col)

    # Add implicit production of allocated dataset
    data[-1] = 1
    indices[-1] = (dataset.id, dataset.id)
                
    # Note: This assumes everything is in technosphere, a real function would also
    # patch the biosphere
    allocated = bwp.create_datapackage(sum_intra_duplicates=True, sum_inter_duplicates=False)
    allocated.add_persistent_vector(
        matrix="technosphere_matrix",
        indices_array=indices,
        flip_array=flip,
        data_array=data,
        name=f"Allocated version of {dataset}",
    )
    return allocated

In [43]:
dp = economic_allocation(bd.get_activity(("example mf", "1")))

In [44]:
lca = bc.LCA({bd.get_id(("example mf", "2")): 1}, data_objs=data_objs + [dp])

In [45]:
lca.lci()

Note that the last two columns, when summed together, form the unallocated activity (column 4):

In [46]:
lca.technosphere_matrix.toarray()

array([[ 1. ,  0. , -2. , -2. , -0.5, -1.5],
       [ 0. ,  1. , -4. , -4. , -1. , -3. ],
       [ 0. ,  0. ,  1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  2. ,  2. ,  0. ],
       [ 0. ,  0. ,  0. ,  4. ,  0. ,  4. ]])

To make sure what we have done is clear, we can create the matrix just for the "delta" data package:

In [47]:
mu.MappedMatrix(packages=[dp], matrix="technosphere_matrix").matrix.toarray()

array([[ 0. , -0.5, -1.5],
       [ 0. , -1. , -3. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  2. ,  0. ],
       [ 0. ,  0. ,  4. ]])

And we can now do LCAs of both allocated products:

In [48]:
lca.lcia()
lca.score

5.25

In [49]:
lca = bc.LCA({bd.get_id(("example mf", "3")): 1}, data_objs=data_objs + [dp])
lca.lci()
lca.lcia()
lca.score

7.875